In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
import threading
import multiprocessing

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs

from tensorflow.keras.layers import InputLayer, BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as keras
import random



from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score


In [2]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np



In [3]:
path = "./data/"
trainX_raw, trainY_raw = load_data("", "train", path)
valX_raw, valY_raw = load_data("", "Val", path)

#trainX_balanced, trainY_balanced = get_balanced_set(trainX_raw, trainY_raw)

valX, valY = valX_raw, valY_raw

features = trainX_raw.shape[-1]


In [4]:
#print(Counter(trainY))

time_steps = 20

train_x_seq, train_y_seq = create_dataset(trainX_raw, trainY_raw , time_steps=time_steps, null_value = 0)
val_x_seq, val_y_seq = create_dataset(valX, valY, time_steps=time_steps, null_value = 0)

#print(Counter(train_y_seq))

X_over, y_over = get_balanced_set_seq(train_x_seq, train_y_seq)

#print(Counter(y_over))

normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(time_steps, features))
normalizer.adapt(X_over)

print("{} {} | {} {}".format(X_over.shape, y_over.shape, val_x_seq.shape, val_y_seq.shape))


(26510, 20, 26) (26510,) | (2434, 20, 26) (2434,)


In [5]:

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(100)


def get_balanced_set(x, y):
    oversample = RandomOverSampler(sampling_strategy='minority')
    #print(Counter(y))
    X_over, y_over = oversample.fit_resample(x, y)
    #print(Counter(y_over))
    return X_over, y_over


metric = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

metric = 'accuracy'

In [6]:
%%time


def get_full_model(
                  seqs = time_steps, 
                  features = 3, 
                  layers = 1, 
                  units = 5, 
                  optimizer='Adam',
                  activation='relu',
                  k_reg=regularizers.l2(0.01),
                  init='glorot_uniform',
                  lays_seqs = 3
                 ):

    set_seeds()

    model = Sequential()

    #, input_shape=(seqs, features)
    
    model.add(normalizer)
        
    model.add(Dense(units, 
                  kernel_initializer=init,
                  kernel_regularizer=k_reg,
                  activation=activation))
    

    for lay in range(int(layers)):
        model.add(Dense(units, 
                        kernel_initializer=init, 
                        kernel_regularizer=k_reg,
                        activation=activation))

    model.add(Dropout(0.5))

    model.add(Dense(1, 
                  kernel_initializer=init,
                  kernel_regularizer=k_reg,
                  activation='sigmoid')) 

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=[metric])
    
    return model


def get_rnn_model(
                  seqs = time_steps, 
                  features = 3, 
                  layers = 1, 
                  units = 5, 
                  optimizer='Adam',
                  activation='relu',
                  k_reg=regularizers.l2(0.01),
                  init='glorot_uniform',
                  lays_seqs = 3
                 ):

    set_seeds()

    model = Sequential()

    #, input_shape=(seqs, features)
    
    model.add(normalizer)
        
    model.add(Dense(units, 
                  kernel_initializer=init,
                  kernel_regularizer=k_reg,
                  activation=activation))
    
    for lay in range(int(lays_seqs)):
        model.add(LSTM(units, kernel_initializer=init, return_sequences=True))
        #model.add(GRU(units, kernel_initializer=init, return_sequences=True))
    
    #x = Bidirectional(LSTM(dim, return_sequences=False))(x)
    model.add(LSTM(units, kernel_initializer=init, return_sequences=False))
    #model.add(GRU(units, kernel_initializer=init, return_sequences=False))


    for lay in range(int(layers)):
        model.add(Dense(units, 
                        kernel_initializer=init, 
                        kernel_regularizer=k_reg,
                        activation=activation))

    model.add(Dropout(0.5))

    model.add(Dense(1, 
                  kernel_initializer=init,
                  kernel_regularizer=k_reg,
                  activation='sigmoid')) 

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=[metric])
    
    return model

#model = get_rnn_model(**params)
#model.summary()

CPU times: user 90 µs, sys: 15 µs, total: 105 µs
Wall time: 119 µs
CPU times: user 8.08 ms, sys: 1.54 ms, total: 9.62 ms
Wall time: 8.58 ms


In [ ]:
%%time

# Get_full_model

from tensorflow.keras import layers
from tensorflow.keras import regularizers



model = KerasClassifier(build_fn=get_rnn_model, 
                        validation_data = (val_x_seq, val_y_seq),
                        verbose=0
                       )
# grid search epochs, batch size and optimizer
optimizers = [
              "Adadelta",
              "Nadam"
              #, "Adam"
]

init = [
        'glorot_uniform', 
        #, 'normal' 
        'he_uniform'
        ]

activation = [
        #'softmax', 
        #'softplus', 
        #'softsign', 
        'relu', 
        'tanh', 
        #'sigmoid', 
        #'hard_sigmoid', 
        #'linear'
        ]

epochs = [100]
batches = [64]
features = [features]
seqs = [time_steps]
layers = [15, 20]
units = [162]
regs = [ "l2", "l1"]
lays_seqs = [5, 10]

#get_simple_model(features[0]).summary()

ea = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    restore_best_weights=True
)

param_grid = dict(
                  seqs = seqs,
                  optimizer=optimizers, 
                  epochs=epochs, 
                  batch_size=batches,
                  layers = layers,
                  units = units,
                  features = features,
                  lays_seqs = lays_seqs,
                  k_reg=regs,
                  activation = activation
                  )

grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid,
                    verbose=4,
                    cv=2,
                    #scoring = 'average_precision'
                    #n_jobs = -1
                   )

grid_result = grid.fit(X_over, y_over, callbacks=[ea])



Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV 1/2] END activation=relu, batch_size=64, epochs=100, features=26, k_reg=l2, layers=15, lays_seqs=5, optimizer=Adadelta, seqs=20, units=162;, score=0.499 total time=12.4min
[CV 2/2] END activation=relu, batch_size=64, epochs=100, features=26, k_reg=l2, layers=15, lays_seqs=5, optimizer=Adadelta, seqs=20, units=162;, score=0.501 total time=11.8min
[CV 1/2] END activation=relu, batch_size=64, epochs=100, features=26, k_reg=l2, layers=15, lays_seqs=5, optimizer=Nadam, seqs=20, units=162;, score=0.499 total time=12.7min
[CV 2/2] END activation=relu, batch_size=64, epochs=100, features=26, k_reg=l2, layers=15, lays_seqs=5, optimizer=Nadam, seqs=20, units=162;, score=0.501 total time=11.5min
[CV 1/2] END activation=relu, batch_size=64, epochs=100, features=26, k_reg=l2, layers=15, lays_seqs=10, optimizer=Adadelta, seqs=20, units=162;, score=0.499 total time=17.4min
[CV 2/2] END activation=relu, batch_size=64, epochs=100, feature

[CV 2/2] END activation=tanh, batch_size=64, epochs=100, features=26, k_reg=l2, layers=20, lays_seqs=10, optimizer=Nadam, seqs=20, units=162;, score=0.501 total time=15.3min
[CV 1/2] END activation=tanh, batch_size=64, epochs=100, features=26, k_reg=l1, layers=15, lays_seqs=5, optimizer=Adadelta, seqs=20, units=162;, score=0.499 total time= 8.1min
[CV 2/2] END activation=tanh, batch_size=64, epochs=100, features=26, k_reg=l1, layers=15, lays_seqs=5, optimizer=Adadelta, seqs=20, units=162;, score=0.501 total time= 8.0min
[CV 1/2] END activation=tanh, batch_size=64, epochs=100, features=26, k_reg=l1, layers=15, lays_seqs=5, optimizer=Nadam, seqs=20, units=162;, score=0.499 total time= 8.2min
[CV 2/2] END activation=tanh, batch_size=64, epochs=100, features=26, k_reg=l1, layers=15, lays_seqs=5, optimizer=Nadam, seqs=20, units=162;, score=0.501 total time= 8.3min
[CV 1/2] END activation=tanh, batch_size=64, epochs=100, features=26, k_reg=l1, layers=15, lays_seqs=10, optimizer=Adadelta, seq

In [ ]:

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

params = grid_result.best_params_
del params['batch_size']
del params['epochs']

In [ ]:
%%time
model = get_rnn_model(**params)


model.fit(
          X_over, y_over,
          epochs=100, 
          verbose=False,
          batch_size=64,
          callbacks=[ea],
          validation_data = (val_x_seq, val_y_seq),
          )

#model.summary()

In [ ]:
res = pd.DataFrame(model.history.history)
res[[metric, 'val_' + metric]].plot(figsize=(10, 6), style='--');

In [ ]:
from sklearn.metrics import classification_report

def eval_data(model, x, y):

    print(model.evaluate(x, y))

    pred = np.where(model.predict(x) > 0.5,1,0)

    print("Confusion Matrix {} {}".format(pred.shape, y.shape))
    print(confusion_matrix(y, pred))



In [ ]:
print("Train")
#print(eval_data(model, training_data[cols], training_data['direction']))
print(eval_data(model, X_over, y_over))
print("Test")
#print(eval_data(model, test_data[cols], test_data['direction']))
print(eval_data(model, val_x_seq, val_y_seq))


